## Архитектуры CNN

Без функций активации все нейронки были бы просто эквивалентны линейной модели (ну, с некоторыми оговорками по поводу других слоев)

**1)**

<img src="./Alex.png" width="700">

в первом слое свертка ядром 11на11 со страйдом 4

- Свертка ядром 1на1 зачем: (если из 384 получаем 256, то как 2д если мыслить: для каждого канала применяем свои 256 ядер: для 1 канала свои 256, для 2 свои 256 и тд, затем, все что для своего 1 ядра: суммируем - 1 карта активации из 256 получается, и так делаем со всеми 256 ядрами)

1) важно, что применяется функция активации и мы добавляем нелинейность (сверточный слой - это простое линейное преобразвание, поэтому без функции активаии две свертки эквиваленты какой-то одной свертке)

2) можем получить другое количество карт активации

Достижения AlexNet:

- использование релу
- параллельное обучение на нескольких карточках
- Data Augmentation

**2)**

**VGG:**

- Сеть более глубокая, больше FC слоев, больше сверточных слоев (аж 19 в VGG19)
- Ядра 3на3 были использованы, то есть более локальные/детальные паттерны могли быть замечены, потому что уже не область 11на11 делалась в 1 пиксель, а область всего 3на3

### Затухание градиентов

Проблема характерная для очень глубоких сетей

<img src="./gr_dec.png" width="700">

Пока градиент идет до самых первых весов, он затухает, а если сетка очень глубокая, то входные весы могут даже и не начать изменяться

- сигмоида - плохо, потому что производная сигмоиды почти не отличается от 0 вдали от x=0, что еще больше все усугубляет

#### Skip connection - решение

<img src="./skip.png" width="700">

$x_3 = x_2 w_2 + x_1$ - x1 связываем с x3, перекидывая его без веса, тогда при дифференцировании $dx_3/dw_0$ будет уже в виде двух слагаемых, потому что теперь x3 зависит от двух предыдущих слоев

Ну и таким образом мы на несколько порядков увеличиваем нужные нам градиенты в ранних слоях и все веса дальше будут хорошо обучаться

<img src="./skip2.png" width="400">

Мы просто на выход получаем H = F(x) + X_input,  $dL/dw_0 = dL/dH (dH/dF *...+dH/dX_{input})=0.1(0.1*0.1 + 1)$ - во внутрь просто единичка добавляется и просто на 2 порядка увеличивается производная лосса

Нам надо к слою перед выходом прибавить просто самый стартовый вектор признаков

In [4]:
import torch

def forward(self, X: torch.Tensor) -> torch.Tensor:
    '''Принцип скип коннекшн на примере двух сверточных слоев (одного сверточного блока):'''
    identity = X        # сохраняем входной вектор, чтобы к самому выходу его прибавить
    out = self.conv1(X)            # создаем out, чтобы случайно X глобальный не изменить и нормально сделать skip connection
    out = self.bn1(out)
    out = self.relu(out)

    out = self.conv2(out)
    out = self.bn2(out)         # надо юзать два разных батч норм слоя, естественно, потому что там все характеристики разные

    if self.downsample is not None:
        identity = self.downsample(X)

    out += identity         # На самом деле, после какого момента мы хотим сделать skip connection - 
                            # решать нам - это надо сделать, когда градиенты уже затухли в обратном направлении.
    out = self.relu(out)

    return out    

Естесна можно делать скип коннекшн несколько раз, сохраняя вектор, который прибавлять на разных этапах или все время один прибавляя

Чаще всего он используется в CNN, 

- также он помогает в поздние слои сети протолкнуть информацию из ранних слоев

**3)**

<img src="./res.png" width="800">

Здесь представлены варианты одного FC блока для resnet - можно по разному ставить батч норм, прибавление коннекшн вектора и функцию активации для передачи в следующий блок

Обычно скип коннекшн юзают как раз по окончании блока (Dense block - как раз составные части):
- сверточный блок - это то что сверху forward - два слоя свертки и потом прибавка вектора на входе в блок
  
- FC блок - сверху на картинке

Сама ResNet:

<img src="./res2.png" width="800">

стрелочки - как раз скип коннекшн перед выходом из блока - на входе в блок сохраняем входной вектор, потом им скипаем все преобразования, а с другим таким же вектором делаем все преборазования - перед выходом их складываем

У нас теперь еще появляется некоторый обходной путь, как данныые могут не напрямую попадать в следующие слои сетки

##### DenseNet

каждый последующий слой сети получает на вход все выходы вообще всех предыдущих сетей: https://youtu.be/TcUPuKpIlhQ?t=2049

- Таким образом каждый следующий слой получает все карты активации всех предыдущих слоев - он получает как низкоуровневую информацию, так и высокоуровневую

<img src="./dense.png" width="800">

Зеленый полчает все фича мапы с красного плюс еще создает свои 4 новые на основе полученных красных. Фиолетовый получает все с красного, зеленого и еще свои 4 создает на основе красных и зеленых, и так далее - каждый следующий слой строит новые gowth_rate карт

потом просто в FC слои передаем финальный вектор, сконкатенировав выход

<img src="./dense2.png" width="800">

каждый блок состоять может из разного количества сверток - классический вариант две свертки 3на3 со всеми наворотами

1на1 свертку юзаем, чтобы новых ровно growth_rate карт активации получать

- Очевидно, что на вот этих вот слоях свертки в dense net нам не надо юзать skip connection как-то отдельно, потому что здесь мы итак передаем вообще все с предыдущих слоев и градиент вообще не будет затухать - в этом главный плюс такой архитектуры

- Более далекие слои таким образом могут паттерны и low level и high level замечать, что позволяет ей обучаться на не очень больштх данных

Можно использовать разные архитектуры блоков:
- basic - две 3на3 свертки 
- bottleneck - 1на1 свертка, 3на3 свертка, 1на1 свертка
- wide
- pyramidal - размеры карт активации постепенно увеличиваются
- pyramidal bottleneck

ModileNet - оптимизирована для запуска с телефона

#### Inception

на этом GoogleNet основана:

мы параллельно слой прогоняем через свертки разных размеров и через разные пуллинги и потом конкатенируем

https://youtu.be/TcUPuKpIlhQ?t=2932 -объяснение GoogleNet

Кратко: мы в разные части сети ставим FC слои и минимизируем сумму лоссов для всех таких выходов, за счет чего у нас не будут затухать градиенты, потому что 1 FC блок будет близко к началу, второй будет близко к середине, третий будет близко к концу

То есть skip connection не единственное возможное решение затухания градиентов

---

## Transfer Learning

Что делать если датасет содержит мало объектов, а обучать модель надо. Если объектов мало, то модель очень легко переобучить, она банально выучит всю выборку эту и будет давать на ней хорошую точность, а на любых других данных она еще ниче не умеет делать

**Fine tuning (дообучение)**

Пусть у нас есть обученная ResNet для классификации картинок на ImageNet на 1000 классов

Мы хотим сетку, которая будет классифицировать диких животных, но изображений с ними у нас очень мало

Обучать с нуля глупо - модель просто переобучится

- Просто инициализируем весы нашей сетки весами из обученной на ImageNet сетки

- было 1000 выходных нейронов, теперь их 10, возьмем и просто выкинем последний слой и весы в нем случайным образом инициализируем

Но так все все равно переобучиться, мы просто инициализируем теперь не рандомами, а какими-то новыми весами другой сетки.

- Нужно заморозить весы первых слоев, потому что там выделяются low level паттерны и по-сути там ничего особо обучаться и не будет, таким образом у нас будет обучаться гораздо меньше параметров

- можно только 2 последних не замораживать, а можно как-то по-другому, это все зависит от различия между датасетами, от объема датасета

- Чем больше различие между датасетами и чем меньше размер датасета для target задачи, тем больще слев надо будет дообучать

<img src="./tl.png" width="600">

Модель выучивает распределение на лейблы при условии первой выборки, а мы хотим получить распределение на лейблы при услвоии второй выборки

TL делится на 2 или 3 части:

на 2:
1) Если разные домены (датасеты) - классифицировать цифры, но у source был датасет MNIST, а мы хотим на SVHN цифры классифицировать
2) Разные задачи - оба датасета состоят из лиц людей, но в одном случае надо определять пол, а в другом расу по лицу

на 3 вида: Это связано с тем, что: $P(Y|D)=\dfrac{P(D|Y)P(Y)}{P(D)}$, здесь P - это именно распределение
1) Ds $\ne$ Dt
   
2) P(Ds) $\ne$ P(Dt), то есть распределения признаков в доменах разные, то есть они например состоят оба из слов на русском языке, но в одном у нас больше слов про кино, в другом про игры.
   
3) P(Ys|Ds) $\ne$ P(Yt|Dt) - то есть у нас совпадают датасеты - оба из лиц людей, но лейблы разные: в первом случае эмоцию определить, во втором пол

- Деление supervised/unsupervised: есть ли лейблы у Dt

То есть мы хотим решать абсолютно такую же задачу, но в target domain у нас нет размеченных лейблов, у нас есть только признаки

#### Как решать эти задачи transfer learning?

- inter-domain информация - характерна обеим датасетам

- intro-domain - информация характерная только target датасету

1) Идея: fine tuning

2) Идея: если у нас нет лейблов на таргет домене, то можно как-то сделать так, что распределения выходов каждого слоя нашей известной сетки было таким же как распределение выходов для таргет сетки

3) Идея: Разбить сеть на много разных частей и пробовать выучивать то, что характерно для одного и другого датасетов 

4) Идея - использовать выходы известной сети и с помощью лосса как-то сближать выходы нашей таргет сети

- Если мы просто будем генерировать данные, то у нас не будут совпадать распределения $P(D_s)$ - то что мы нагенерировали и $P(D_t)$ - какое есть распределение для реальных данных для объектов, которые мы генерировали. Если мы начнем просто обучать сетку на $P(D_t)$, то в реальном мире модель будет плохо работать, поэтому надо применять transfer learning для модели, которую мы обучили на сгенерированных данных, а потом уже на нашей какой-то маленькой выборке из реального мире обучать модель для реального мира